In [1]:
print ("ok")

ok


In [2]:
%pwd

'c:\\Euron\\medical_chatbot-GenerativeAI\\research'

In [3]:
import os
os.chdir ("../")

In [4]:
%pwd

'c:\\Euron\\medical_chatbot-GenerativeAI'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader


In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [7]:
def load_pdf_file(data):
    loader = DirectoryLoader(data, glob = "*.pdf",
                             loader_cls = PyPDFLoader)
    documents = loader.load()
    return documents 

In [8]:
%pwd

'c:\\Euron\\medical_chatbot-GenerativeAI'

In [9]:

extracted_data = load_pdf_file(data = 'data')

c:\Users\ell.DESKTOP-F8FOOJM\AppData\Local\Programs\Python\Python311\Lib\site-packages\pypdf\_crypt_providers\_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4


In [10]:
#extracted_data

In [11]:
def text_chunks (extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunk = text_splitter.split_documents(extracted_data)
    return text_chunk 

In [12]:
text_splits = text_chunks (extracted_data)
print(" length of text chunks ", len(text_splits))

 length of text chunks  7020


In [13]:
from langchain.embeddings import HuggingFaceEmbeddings

In [14]:
def download_huggingface_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [15]:
embeddings = download_huggingface_embeddings()

c:\Users\ell.DESKTOP-F8FOOJM\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:

query_results = embeddings.embed_query("Hello world")
print("length of query: ", len(query_results))



length of query:  384


In [31]:
from dotenv import load_dotenv
load_dotenv()




True

In [32]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
GROQ_API_KEY = os.environ.get('GROQ_API_KEY')

In [33]:
import os
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY
os.environ['GROQ_API_KEY'] = GROQ_API_KEY

In [ ]:
#create the index for Pinecone manually 
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

pc = Pinecone(api_key = PINECONE_API_KEY)
index_name =  "testbot1"
pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

In [23]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    
    documents= text_splits,
    index_name = index_name,
    embedding = embeddings
)

In [24]:
#load the existing_index
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name = index_name,
    embedding = embeddings
)

In [25]:
docsearch 

In [26]:
retriever = docsearch.as_retriever(search_type ="similarity", search_kwargs = {"k":3})

In [27]:
retriever_docs = retriever.invoke("what is acne")

In [28]:
retriever_docs

[Document(page_content='Acidosis seeRespiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when thepores of the skin become clogged with oil, dead skincells, and bacteria.\nDescription\nAcne vulgaris, the medical term for common acne, is', metadata={'page': 37.0, 'source': 'data\\Medical_book (1).pdf'}),
 Document(page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25Acne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceousglands become inflamed. (Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)GEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25', metadata={'page': 38.0, 'source': 'data\\Medical_book (1).pdf'}),
 Document(page_content='ent purposes. For example, lotions, soaps, gels, andcreams containing benzoyl peroxide or tretinoin may beused to clear up mild to moder

In [34]:
from langchain_groq import ChatGroq
llm = ChatGroq(temperature = 0.4, max_tokens = 500, model="llama3-8b-8192")


In [35]:
from langchain.chains import create_retrieval_chain
from langchain. chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [36]:
system_promt  = (
                 
                 "you are an assistant for question answering tasks."
                 "use the following pieces of retrieved context to answer"
                 "the question. If you don't know the answer, say that you"
                 "don't know. use the 3 sentenced maximum and keep the answer concise."
                 "\n\n"
                "{context}"
                 
                 
                 
                 
                 
)

In [40]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_promt),
        ("human", "{input}")
    ]
)

In [41]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [45]:
response = rag_chain.invoke({"input": "what is acne"})

In [46]:
print(response["answer"])

Acne is a common skin disease characterized by pimples on the face, chest, and back, caused by clogged pores with oil, dead skin cells, and bacteria.
